In [1]:
#get daily S&P 500 closing price
#https://codingandfun.com/python-for-finance-calculate-and-plot-sp-500-daily-returns/
import pandas as pd
#if you get an error after executing the code, try adding below:
pd.core.common.is_list_like = pd.api.types.is_list_like

#import necessary modules
import pandas_datareader.data as web
import datetime
from datetime import datetime

#create start and end times
start = datetime.date(datetime(2020,1,2))
end = datetime.date(datetime.now())

#get the SP500 index information
SP500 = web.DataReader(['sp500'], 'fred', start, end)
SP500.head()

,sp500
DATE,
2020-01-02,3257.85
2020-01-03,3234.85
2020-01-06,3246.28
2020-01-07,3237.18
2020-01-08,3253.05


In [2]:
#copy the data to make changes to the data frame
SP500_data = SP500.copy()

#reset the index so that date is it's own column
SP500_data.reset_index(level=0, inplace=True)

#display results
SP500_data.head()

,DATE,sp500
0,2020-01-02,3257.85
1,2020-01-03,3234.85
2,2020-01-06,3246.28
3,2020-01-07,3237.18
4,2020-01-08,3253.05


In [3]:
#convert to string for upload
SP500_data['DATE'] = SP500_data['DATE'].dt.strftime('%Y-%m-%d')

#check results
SP500_data.head()

,DATE,sp500
0,2020-01-02,3257.85
1,2020-01-03,3234.85
2,2020-01-06,3246.28
3,2020-01-07,3237.18
4,2020-01-08,3253.05


In [4]:
#get week ending date
from datetime import datetime, timedelta
from pandas.tseries.offsets import *

#calculate new columns for week ending
SP500_data['DATE_obj'] = pd.to_datetime(SP500_data['DATE'])
#SP500_data['week_end'] = SP500_data['DATE_obj'] + Week(weekday=5)
SP500_data['week_ending'] = SP500_data['DATE_obj'].where(SP500_data['DATE_obj'] == ((SP500_data['DATE_obj'] + Week(weekday=5) ) - Week()), SP500_data['DATE_obj'] + Week(weekday=5))

#check results
SP500_data.head()

,DATE,sp500,DATE_obj,week_ending
0,2020-01-02,3257.85,2020-01-02,2020-01-04
1,2020-01-03,3234.85,2020-01-03,2020-01-04
2,2020-01-06,3246.28,2020-01-06,2020-01-11
3,2020-01-07,3237.18,2020-01-07,2020-01-11
4,2020-01-08,3253.05,2020-01-08,2020-01-11


In [5]:
#now convert the Week Ending back into a string for compliance with other datasets
SP500_data['week_ending'] = SP500_data['week_ending'].dt.strftime('%Y-%m-%d')

#check results
SP500_data.head(20)

,DATE,sp500,DATE_obj,week_ending
0,2020-01-02,3257.85,2020-01-02,2020-01-04
1,2020-01-03,3234.85,2020-01-03,2020-01-04
2,2020-01-06,3246.28,2020-01-06,2020-01-11
3,2020-01-07,3237.18,2020-01-07,2020-01-11
4,2020-01-08,3253.05,2020-01-08,2020-01-11
5,2020-01-09,3274.70,2020-01-09,2020-01-11
6,2020-01-10,3265.35,2020-01-10,2020-01-11
7,2020-01-13,3288.13,2020-01-13,2020-01-18
8,2020-01-14,3283.15,2020-01-14,2020-01-18
9,2020-01-15,3289.29,2020-01-15,2020-01-18


In [6]:
#now send this data to SQL!
#reference: https://stackoverflow.com/questions/30631325/writing-to-mysql-database-with-pandas-using-sqlalchemy-to-sql
import pymysql
from sqlalchemy import create_engine

user = 'admin'
passw = 'dude1990'
host =  'group3final.cx1foeolbgvg.us-east-2.rds.amazonaws.com'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
port = 3306 
database = 'stock_data'

mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

SP500_data.to_sql(name='sp_data', con=mydb, if_exists = 'replace', index=False)